In [1]:
import sys
sys.path.append('../OOSpotify/')
from OOSpotify import *
import pickle
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
def prepSequenceData(track,maxlen=4163):
    aa = track.getAudioAnalysis()
    df = pd.DataFrame(aa['segments'])
    pitches = np.array([i for i in df.pitches])
    timbres = np.array([i for i in df.timbre])
    loudness = np.repeat(df.loudness_max.tolist(),12).reshape(-1,12)
    duration = np.repeat(df.duration.tolist(),12).reshape(-1,12)
    confidence = np.repeat(df.confidence.tolist(),12).reshape(-1,12)
    sequence_data = np.concatenate((pitches,timbres,loudness,duration,confidence),axis=1)
    sequence_data_padded = pad_sequences([sequence_data],maxlen=maxlen)
    return sequence_data_padded

def prepMetaData(track):
    features = track.features.items()
    metadata = np.array([val for key,val in features]).reshape(-1,1)
    return metadata

In [3]:
def extractTracks(SpotifyObjs):
    if type(SpotifyObjs) is not list:
        SpotifyObjs = [SpotifyObjs]
    trackList = []
    for obj in SpotifyObjs:
        if obj.type == 'track':
            trackList.append(obj)
        elif obj.type == 'artist':
            tracks = obj.getTopTracks()
            trackList += tracks
            print('Using top {} tracks for artist {}'.format(len(tracks),obj.name))
        elif obj.type in ['playlist','album']:
            tracks = obj.getTracks()
            trackList += tracks
        else:
            #invalid type
            pass
    return trackList
        
def prepData(SpotifyObjs,verbose=True,mainScaler=None,auxScaler=None):
    seq = []
    meta = []
    tracks = extractTracks(SpotifyObjs)
    trackList = []
    for i,track in enumerate(tracks):
            trackList.append([i,track.name,track.artist])
            if verbose:
                print('{}. {} -- {}'.format(trackList[i][0],trackList[i][1],trackList[i][2]))
            seq.append(prepSequenceData(track))
            meta.append(prepMetaData(track))
    seq = np.squeeze(np.array(seq))
    if seq.ndim == 2:
        seq = np.expand_dims(seq,axis=0)
    meta = np.array(meta).reshape(-1,12)
    if mainScaler:
        seq = mainScaler.transform(seq.reshape(-1,60)).reshape(seq.shape)
    if auxScaler:
        meta = auxScaler.transform(meta)
    print()
    print('Sequence Shape: {}'.format(seq.shape))
    print('Metadata Shape: {}'.format(meta.shape))
    return seq,meta,trackList

In [4]:
genreList = pickle.load(open('genreList.p','rb'))
for i,g in enumerate(genreList):
    print(i,g)

0 Alternative Rock
1 Bluegrass
2 Blues
3 Classical
4 Country
5 Dubstep
6 EDM
7 Electronic
8 Emo
9 Folk
10 Gospel
11 Grunge
12 Hip Hop
13 Indie Rock
14 Jazz
15 Metal
16 Motown
17 Pop
18 Rap
19 Reggae
20 Rock
21 Ska Punk
22 Soundtrack


In [15]:
model = load_model('/data/home/zrachlin/spotifire/Genre_Prediction/bs60.48--2.01.hdf5')

In [52]:
seq,meta,tracklist = prepData([Album('mr. finish line'),Track('back pocket')],mainScaler=min_max_scaler,auxScaler=aux_min_max_scaler)

0. Birds of a Feather, We Rock Together (feat. Antwaun Stanley) -- Vulfpeck
1. Baby I Don’t Know Oh Oh (feat. Charles Jones) -- Vulfpeck
2. Mr. Finish Line (feat. Christine Hucal & Theo Katzman) -- Vulfpeck
3. Tee Time -- Vulfpeck
4. Running Away (feat. Joey Dosik, David T. Walker & James Gadson) -- Vulfpeck
5. Hero Town (feat. Michael Bland) -- Vulfpeck
6. Business Casual (feat. Coco O.) -- Vulfpeck
7. Vulf Pack -- Vulfpeck
8. Grandma (feat. Antwaun Stanley, David T. Walker & James Gadson) -- Vulfpeck
9. Captain Hook (feat. Baby Theo, Bootsy Collins & Mushy Kay) -- Vulfpeck
10. Back Pocket -- Vulfpeck

Sequence Shape: (11, 4163, 60)
Metadata Shape: (11, 12)


In [ ]:
model = load_model('weights.05--2.44.hdf5')

In [39]:
a = model.predict(x=[seq,meta])

In [41]:
['{} --> {}'.format(tracklist[i],genreList[np.argmax(a[0][i])]) for i in range(11)]

['Birds of a Feather, We Rock Together (feat. Antwaun Stanley) --> Folk',
 'Baby I Don’t Know Oh Oh (feat. Charles Jones) --> Pop',
 'Mr. Finish Line (feat. Christine Hucal & Theo Katzman) --> Hip Hop',
 'Tee Time --> Electronic',
 'Running Away (feat. Joey Dosik, David T. Walker & James Gadson) --> Hip Hop',
 'Hero Town (feat. Michael Bland) --> Electronic',
 'Business Casual (feat. Coco O.) --> Hip Hop',
 'Vulf Pack --> Electronic',
 'Grandma (feat. Antwaun Stanley, David T. Walker & James Gadson) --> Hip Hop',
 'Captain Hook (feat. Baby Theo, Bootsy Collins & Mushy Kay) --> Hip Hop',
 'Back Pocket --> Pop']

In [8]:
seq,meta,tracklist = prepData([Album('an awesome wave')])

0. Intro -- alt-J
1. Interlude 1 -- alt-J
2. Tessellate -- alt-J
3. Breezeblocks -- alt-J
4. Interlude 2 -- alt-J
5. Something Good -- alt-J
6. Dissolve Me -- alt-J
7. Matilda -- alt-J
8. Ms -- alt-J
9. Fitzpleasure -- alt-J
10. Interlude 3 -- alt-J
11. Bloodflood -- alt-J
12. Taro -- alt-J
13. Hand-Made -- alt-J

Sequence Shape: (14, 4163, 60)
Metadata Shape: (14, 12)


In [55]:
a = model.predict(x=[seq,meta])
b = ['{} --> {}'.format(tracklist[i],genreList[np.argmax(a[0][i])]) for i in range(len(a[0]))]
print(b)

['Intro --> Electronic', 'Interlude 1 --> Hip Hop', 'Tessellate --> Pop', 'Breezeblocks --> Indie Rock', 'Interlude 2 --> Classical', 'Something Good --> Folk', 'Dissolve Me --> EDM', 'Matilda --> Indie Rock', 'Ms --> Folk', 'Fitzpleasure --> Indie Rock', 'Interlude 3 --> Classical', 'Bloodflood --> Indie Rock', 'Taro --> Folk', 'Hand-Made --> Folk']


In [61]:
seq,meta,tracklist = prepData([Artist('bob dylan').getAlbums()[0]])

0. Slow Train - Live in London -- Bob Dylan
1. Precious Angel - Live at the Warfield Theatre, San Francisco, CA - November 16, 1979 -- Bob Dylan
2. Gotta Serve Somebody - Live at the Warfield Theatre, San Francisco, CA - November 15, 1979 -- Bob Dylan
3. When He Returns - Take 2 -- Bob Dylan
4. Solid Rock - Live at Golden Hall, San Diego, CA - November 27, 1979 -- Bob Dylan
5. Gonna Change My Way of Thinking - Live at the Orpheum Theater, Memphis, TN - January 31, 1980 -- Bob Dylan
6. Ain't Gonna Go to Hell for Anybody - Live at Le Theatre Saint-Denis, Montreal, Quebec - April 24, 1980 -- Bob Dylan
7. When You Gonna Wake Up? - Live at Drammenshallen, Drammen, Norway - July 9, 1981 -- Bob Dylan
8. Covenant Woman - Take 3 -- Bob Dylan
9. Shot of Love - Outtake -- Bob Dylan
10. Making a Liar Out of Me - Rehearsal -- Bob Dylan
11. City of Gold - Live Nov. 22, 1980 -- Bob Dylan
12. Yonder Comes Sin - Rehearsal -- Bob Dylan
13. Caribbean Wind - Rehearsal with pedal steel -- Bob Dylan
14. Dea

In [62]:
a = model.predict(x=[seq,meta])
for i in range(len(a[0])):
    print('{} --> {},{}'.format(tracklist[i],genreList[np.argmax(a[0][i])],genreList[np.argmax(a[1][i])]))
#b = ['{} --> {},{}'.format(tracklist[i],genreList[np.argmax(a[0][i])],genreList[np.argmax(a[1][i])]) for i in range(len(a[0]))]
#print(b)

Slow Train - Live in London --> Rock,Rock
Precious Angel - Live at the Warfield Theatre, San Francisco, CA - November 16, 1979 --> Rock,Classical
Gotta Serve Somebody - Live at the Warfield Theatre, San Francisco, CA - November 15, 1979 --> Indie Rock,Indie Rock
When He Returns - Take 2 --> Jazz,Motown
Solid Rock - Live at Golden Hall, San Diego, CA - November 27, 1979 --> Pop,Classical
Gonna Change My Way of Thinking - Live at the Orpheum Theater, Memphis, TN - January 31, 1980 --> Motown,Indie Rock
Ain't Gonna Go to Hell for Anybody - Live at Le Theatre Saint-Denis, Montreal, Quebec - April 24, 1980 --> Rock,Indie Rock
When You Gonna Wake Up? - Live at Drammenshallen, Drammen, Norway - July 9, 1981 --> Jazz,Classical
Covenant Woman - Take 3 --> Jazz,Motown
Shot of Love - Outtake --> Motown,Motown
Making a Liar Out of Me - Rehearsal --> Rock,Rock
City of Gold - Live Nov. 22, 1980 --> Folk,Indie Rock
Yonder Comes Sin - Rehearsal --> Rock,Rock
Caribbean Wind - Rehearsal with pedal steel

In [9]:
seq,meta,tracklist = prepData([User(users['zach']).getPlaylists()[14]])

0. Kanye,Beyonce -- Rome Fortune
1. Holy Toledo -- Vundabar
2. Step -- Vampire Weekend
3. Down for the Fifth Time -- Flamingosis
4. Bikes -- Rubblebucket
5. See You Leave -- RJD2
6. I'm the Moon -- Spooky Mansion
7. Clay Pigeons -- Michael Cera
8. Panama -- The Cat Empire
9. I Can't Play Piano, Pt. 1 -- Jon Benjamin - Jazz Daredevil
10. Celebrity Reduction Prayer (feat. Open Mike Eagle & Oddisee) -- Mello Music Group
11. Killer Bees -- Demon Days
12. Homage -- Mild High Club
13. 2 a.m. -- Cool Calm Pete
14. Slow Mode -- Windmills
15. Orange Blossom -- Gardens & Villa
16. Tired Boy -- Concept Collective
17. A Fan's Mail (Tron Song Suite II) -- Thundercat
18. Breezin -- Cody G
19. Otha Fish -- The Pharcyde
20. Control It -- Dwynell Roland
21. Black Coffee -- Kill Bill: The Rapper
22. Look at That Ass -- Marc Rebillet
23. Foxtrot -- Simon Jefferis
24. Starman -- SHT GHST
25. Exclamation Point -- Latyrx
26. Foundation -- KAMI
27. Skywards -- Pick Patek
28. Alaska - Tycho Remix -- Maggie Ro

In [19]:
a = model.predict(x=[seq,meta])
results = [sorted(range(len(a[0][i])),key=lambda j: a[0][i][j],reverse=True)[:3] for i in range(len(a[0]))]
for i in range(len(a[0])):
    print('{}. {} --> 1: {}, 2: {}, 3: {}, Aux: {}'.format(i,tracklist[i],genreList[results[i][0]],genreList[results[i][1]],genreList[results[i][2]],genreList[np.argmax(a[1][i])]))

0. Kanye,Beyonce --> 1: Reggae, 2: Blues, 3: Folk, Aux: Country
1. Holy Toledo --> 1: Metal, 2: Alternative Rock, 3: Emo, Aux: Rock
2. Step --> 1: Hip Hop, 2: Rap, 3: Reggae, Aux: Hip Hop
3. Down for the Fifth Time --> 1: Hip Hop, 2: Reggae, 3: Rap, Aux: Hip Hop
4. Bikes --> 1: Indie Rock, 2: Ska Punk, 3: Alternative Rock, Aux: EDM
5. See You Leave --> 1: Hip Hop, 2: Rap, 3: Pop, Aux: Hip Hop
6. I'm the Moon --> 1: Reggae, 2: Hip Hop, 3: Motown, Aux: Reggae
7. Clay Pigeons --> 1: Indie Rock, 2: Folk, 3: Bluegrass, Aux: Soundtrack
8. Panama --> 1: Indie Rock, 2: Alternative Rock, 3: Country, Aux: Indie Rock
9. I Can't Play Piano, Pt. 1 --> 1: Reggae, 2: Blues, 3: Jazz, Aux: Reggae
10. Celebrity Reduction Prayer (feat. Open Mike Eagle & Oddisee) --> 1: Indie Rock, 2: Hip Hop, 3: Pop, Aux: Electronic
11. Killer Bees --> 1: Indie Rock, 2: Alternative Rock, 3: Country, Aux: Indie Rock
12. Homage --> 1: Indie Rock, 2: Alternative Rock, 3: Rock, Aux: Rock
13. 2 a.m. --> 1: Ska Punk, 2: Reggae

In [21]:
seq,meta,tracklist = prepData([Track('the three of us'),Track('let it be')])
a = model.predict(x=[seq,meta])
results = [sorted(range(len(a[0][i])),key=lambda j: a[0][i][j],reverse=True)[:3] for i in range(len(a[0]))]
for i in range(len(a[0])):
    print('{}. {} --> 1: {}, 2: {}, 3: {}, Aux: {}'.format(i,tracklist[i],genreList[results[i][0]],genreList[results[i][1]],genreList[results[i][2]],genreList[np.argmax(a[1][i])]))

0. The Three of Us -- Streetlight Manifesto
1. Let It Be - Remastered -- The Beatles

Sequence Shape: (2, 4163, 60)
Metadata Shape: (2, 12, 1)
0. The Three of Us --> 1: Ska Punk, 2: Alternative Rock, 3: Indie Rock, Aux: Emo
1. Let It Be - Remastered --> 1: Bluegrass, 2: Country, 3: Folk, Aux: Bluegrass


In [7]:
def predictGenres(seq,meta,trackList,model):
    a = model.predict(x=[seq,meta])
    results = [sorted(range(len(a[0][i])),key=lambda j: a[0][i][j],reverse=True)[:3] for i in range(len(a[0]))]
    for i in range(len(trackList)):
        print('{}. {} ({}) --> 1: {}, 2: {}, 3: {}, Aux: {}'.format(trackList[i][0],trackList[i][1],trackList[i][2],genreList[results[i][0]],genreList[results[i][1]],genreList[results[i][2]],genreList[np.argmax(a[1][i])]))
    return a
def prepandpredictGenres(tracks,model,mainScaler=None,auxScaler=None):
    print('Preparing Data ...')
    seq,meta,trackList = prepData(tracks,verbose=False,mainScaler=mainScaler,auxScaler=auxScaler)
    print()
    a = model.predict(x=[seq,meta])
    results = [sorted(range(len(a[0][i])),key=lambda j: a[0][i][j],reverse=True)[:3] for i in range(len(a[0]))]
    for i in range(len(trackList)):
        print('{}. {} ({}) --> 1: {}, 2: {}, 3: {}, Aux: {}'.format(trackList[i][0],trackList[i][1],trackList[i][2],genreList[results[i][0]],genreList[results[i][1]],genreList[results[i][2]],genreList[np.argmax(a[1][i])]))
    return a

In [10]:
res = prepandpredictGenres([Artist('streetlight manifesto'),Artist('kanye west')],model)

Preparing Data ...
Using top 10 tracks for artist Streetlight Manifesto
Using top 10 tracks for artist Kanye West

Sequence Shape: (20, 4163, 60)
Metadata Shape: (20, 12)

0. The Three of Us (Streetlight Manifesto) --> 1: Ska Punk, 2: Alternative Rock, 3: Emo, Aux: Alternative Rock
1. We Will Fall Together (Streetlight Manifesto) --> 1: Ska Punk, 2: Rock, 3: Alternative Rock, Aux: Ska Punk
2. Everything Went Numb (Streetlight Manifesto) --> 1: Ska Punk, 2: Indie Rock, 3: Rock, Aux: Metal
3. Point / Counterpoint (Streetlight Manifesto) --> 1: Ska Punk, 2: Country, 3: Rock, Aux: Blues
4. A Better Place, a Better Time (Streetlight Manifesto) --> 1: Ska Punk, 2: Emo, 3: Indie Rock, Aux: Ska Punk
5. Would You Be Impressed? (Streetlight Manifesto) --> 1: Ska Punk, 2: Emo, 3: Indie Rock, Aux: Metal
6. Down, Down, Down to Mephisto's Cafe (Streetlight Manifesto) --> 1: Ska Punk, 2: Rock, 3: Blues, Aux: Blues
7. Such Great Heights (Streetlight Manifesto) --> 1: Emo, 2: EDM, 3: Indie Rock, Aux: E

In [14]:
res = prepandpredictGenres(Track('stuck inside of mobile'),model)

Preparing Data ...

Sequence Shape: (1, 4163, 60)
Metadata Shape: (1, 12)

0. Stuck Inside of Mobile with the Memphis Blues Again (Bob Dylan) --> 1: Rock, 2: Alternative Rock, 3: Grunge, Aux: Rock


In [11]:
eva = User('126233477')
eva.Playlists()

0: for mom
1: Flip It
2: My Shazam Tracks
3:  feelin good
4: yup
5: Good Times
6: Baby Makin Music
7: Happy Folk
8: chill, bro
9: comedy
10: sing
11: Change of Scenery
12: +
13: You'll Get There
14: I Nap
15: Lush + Atmospheric
16: Pretty & Pretty Sad
17: Get Ready
18: Calm Happy
19: Get Down Good Lord
20: Ed sheeran, ben howard, Damien Rice, Bon Iver, James Vincent McMorrow, Iron and Wine
21: Run!
22: Chill Out‏‏​​   ‍
23: Liked from Radio
24: Misfits Soundtrack


In [17]:
res = prepandpredictGenres(eva.getPlaylists()[2],model)

Preparing Data ...

Sequence Shape: (32, 4163, 60)
Metadata Shape: (32, 12)

0. Something I Need (OneRepublic) --> 1: Pop, 2: Country, 3: Indie Rock, Aux: Pop
1. Hurt Me Tomorrow (K'NAAN) --> 1: EDM, 2: Pop, 3: Rap, Aux: EDM
2. 'Tain't Nobody's Biz-ness If I Do (Lou Rawls) --> 1: Jazz, 2: Blues, 3: Motown, Aux: Motown
3. Sunshine Lady (Dion) --> 1: Blues, 2: Reggae, 3: Motown, Aux: Blues
4. Closer (The Chainsmokers) --> 1: Pop, 2: Rap, 3: Hip Hop, Aux: Pop
5. I'm Ready (AJR) --> 1: EDM, 2: Pop, 3: Hip Hop, Aux: EDM
6. 19-2000 (Gorillaz) --> 1: Reggae, 2: Hip Hop, 3: Rap, Aux: Hip Hop
7. Town & Country (Bibio) --> 1: Bluegrass, 2: Indie Rock, 3: Folk, Aux: Bluegrass
8. I Second That Emotion (Smokey Robinson & The Miracles) --> 1: Rock, 2: Motown, 3: Country, Aux: Rock
9. Worse Things Than Love (Timeflies) --> 1: Pop, 2: Rap, 3: Hip Hop, Aux: Pop
10. Hummingbird (Miss E) --> 1: Blues, 2: Folk, 3: Jazz, Aux: Blues
11. Don't Stop (Ro Malone) --> 1: Pop, 2: Country, 3: EDM, Aux: Pop
12. Coc

In [18]:
r = prepandpredictGenres(eva.getPlaylists()[11].getTracks()[:20],model)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
Preparing Data ...

Sequence Shape: (20, 4163, 60)
Metadata Shape: (20, 12)

0. Glory (Radical Face) --> 1: Electronic, 2: Indie Rock, 3: Soundtrack, Aux: Indie Rock
1. Start Again (Bishop Allen) --> 1: Electronic, 2: Indie Rock, 3: EDM, Aux: Indie Rock
2. Silver And Gold (Noah And The Whale) --> 1: Pop, 2: Rap, 3: Reggae, Aux: Blues
3. The General Specific (Band of Horses) --> 1: Indie Rock, 2: Blues, 3: Ska Punk, Aux: Blues
4. Missing (Eliza Doolittle) --> 1: Reggae, 2: Blues, 3: Motown, Aux: Reggae
5. Verse (Rhye) --> 1: Jazz, 2: Pop, 3: Blues, Aux: Hip Hop
6. The Fall (Rhye) --> 1: Electronic, 2: Jazz, 3: Bluegrass, Aux: Jazz
7. Major Minor Love (Rhye) --> 1: Pop, 2: Folk, 3: Soundtrack, Aux: Pop
8. Sweet Talk (Jessie Ware) --> 1: Rap, 2: Pop, 3: Hip Hop, Aux: Pop
9. Heavenly (Vacationer) --> 1: Pop, 2: Indie Rock, 3: Gospel, Aux: Indie Roc

In [12]:
a,b,c = prepData(eva.getPlaylists()[11])

retrying ...1secs
retrying ...1secs
retrying ...1secs
0. Glory -- Radical Face
1. Start Again -- Bishop Allen
2. Silver And Gold -- Noah And The Whale
3. The General Specific -- Band of Horses
4. Missing -- Eliza Doolittle
5. Verse -- Rhye
6. The Fall -- Rhye
7. Major Minor Love -- Rhye
8. Sweet Talk -- Jessie Ware
9. Heavenly -- Vacationer
10. Unbelievers -- Vampire Weekend
11. Let It Rain -- Mat Kearney
12. I Choose You -- Sara Bareilles
13. Don't Leave Me [Ne Me Quitte Pas] -- Regina Spektor
14. Valentine -- Jessie Ware & Sampha
15. Wandering Star -- POLIÇA
16. A Minute or Two -- Cider Sky
17. What An Experience -- Janelle Monáe
18. Silent Way -- Milo Greene
19. Wonderful Unknown -- Ingrid Michaelson
20. Done Treading Water -- The Speedbumps
21. Strikes and Gutters -- The Speedbumps
22. Opposites in Love -- The Speedbumps
23. Fold -- Kwesi
24. The Switch -- Emily King
25. Weight In Gold -- Gallant
26. Tonight -- Matt Stansberry & the Romance
27. Green & Gold -- Lianne La Havas
28. E

231. For Love, From The Law -- Humeysha
232. Why Don't You Listen -- James Cherry
233. Giuseppe -- Trash Panda
234. Found -- Seramic
235. Rescue -- Alysha Brilla
236. Help -- Weaves
237. Kidnap Me -- CRUISR
238. Strawberry Wine -- No Suits
239. Better -- Samm Henshaw
240. American Privilege -- Allen Stone
241. Turn Around -- Justin Jay
242. Say It's Right -- Gabriel Royal
243. Sing to You -- John Splithoff
244. Can We Work It Out -- Gordi
245. Lonely Lullabies -- Kweku Collins
246. Ultralight Beam -- Emir Taha
247. Lazer Eyes -- Teenage Love
248. Take It All Back 2.0 -- Judah & the Lion
249. Latch -- Moonrise Nation
250. If It Ain't Broke -- Sonny Cleveland
251. Make Out -- Julia Nunes
252. Ten Goodbyes -- Tim Atlas
253. Requiem -- Parlour Tricks
254. Down Home Magic -- Flearoy
255. The Real -- Busty and the Bass
256. Lover Come Back -- City and Colour
257. Arizona -- Frances Cone
258. Cold Little Heart -- Michael Kiwanuka
259. Simple Love -- Isaac and Joel
260. I Want You -- Marian Hi

In [13]:
r = predictGenres(a,b,c,model)

0. Glory (Radical Face) --> 1: Electronic, 2: Indie Rock, 3: Soundtrack, Aux: Indie Rock
1. Start Again (Bishop Allen) --> 1: Electronic, 2: Indie Rock, 3: EDM, Aux: Indie Rock
2. Silver And Gold (Noah And The Whale) --> 1: Pop, 2: Rap, 3: Reggae, Aux: Blues
3. The General Specific (Band of Horses) --> 1: Indie Rock, 2: Blues, 3: Ska Punk, Aux: Blues
4. Missing (Eliza Doolittle) --> 1: Reggae, 2: Blues, 3: Motown, Aux: Reggae
5. Verse (Rhye) --> 1: Jazz, 2: Pop, 3: Blues, Aux: Hip Hop
6. The Fall (Rhye) --> 1: Electronic, 2: Jazz, 3: Bluegrass, Aux: Jazz
7. Major Minor Love (Rhye) --> 1: Pop, 2: Folk, 3: Soundtrack, Aux: Pop
8. Sweet Talk (Jessie Ware) --> 1: Rap, 2: Pop, 3: Hip Hop, Aux: Pop
9. Heavenly (Vacationer) --> 1: Pop, 2: Indie Rock, 3: Gospel, Aux: Indie Rock
10. Unbelievers (Vampire Weekend) --> 1: Electronic, 2: Indie Rock, 3: Alternative Rock, Aux: Electronic
11. Let It Rain (Mat Kearney) --> 1: Pop, 2: Rap, 3: EDM, Aux: Pop
12. I Choose You (Sara Bareilles) --> 1: Blues,

In [5]:
model = load_model('/data/home/zrachlin/spotifire/Genre_Prediction/bs32_minmaxscaled.36--1.69.hdf5')

In [ ]:
need to minmax scale the data...

In [6]:
[min_max_scaler,aux_min_max_scaler] = pickle.load(open('minmaxscalers.p','rb'))

In [55]:
r = prepandpredictGenres(eva.getPlaylists()[11].getTracks()[:20],model,min_max_scaler,aux_min_max_scaler)

retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
Preparing Data ...

Sequence Shape: (20, 4163, 60)
Metadata Shape: (20, 12)

0. Glory (Radical Face) --> 1: Indie Rock, 2: Electronic, 3: Alternative Rock, Aux: EDM
1. Start Again (Bishop Allen) --> 1: Indie Rock, 2: Alternative Rock, 3: Electronic, Aux: Country
2. Silver And Gold (Noah And The Whale) --> 1: Pop, 2: Country, 3: Indie Rock, Aux: Metal
3. The General Specific (Band of Horses) --> 1: Country, 2: Indie Rock, 3: Pop, Aux: Blues
4. Missing (Eliza Doolittle) --> 1: Pop, 2: Hip Hop, 3: Rap, Aux: Gospel
5. Verse (Rhye) --> 1: Folk, 2: Blues, 3: Bluegrass, Aux: Alternative Rock
6. The Fall (Rhye) --> 1: Indie Rock, 2: Alternative Rock, 3: Bluegrass, Aux: Gospel
7. Major Minor Love (Rhye) --> 1: Soundtrack, 2: Pop, 3: Indie Rock, Aux: Gospel
8. Sweet Talk (Jessie Ware) --> 1: Rap, 2: Pop, 3: Hip Hop, Aux: Metal
9. Heavenly (Vacationer) --> 1: Pop, 2: EDM, 3: Gospel, Aux: Metal
10. Unbelievers (Vampire Weekend

In [13]:
pl = [i for i in User('thesoundsofspotify').getPlaylists() if i.name.lower() == 'the sound of rap']

In [15]:
pl[0].name

'The Sound of Rap'

In [16]:
r = prepandpredictGenres(pl,model,min_max_scaler,aux_min_max_scaler)

Preparing Data ...
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs

Sequence Shape: (843, 4163, 60)
Metadata Shape: (843, 12)

0. Bigger Than You (feat. Drake & Quavo) (2 Chainz) --> 1: Rap, 2: Hip Hop, 3: Pop, Aux: Rock
1. Unforgettable (French Montana) --> 1: Rap, 2: Pop, 3: Hip Hop, Aux: Ska Punk
2. Taste (feat. Offset) (Tyga) --> 1: Pop, 2: Rap, 3: Hip Hop, Aux: Alternative Rock
3. Praise The Lord (Da Shine) (A$AP Rocky) --> 1: Hip Hop, 2: Rap, 3: Dubstep, Aux: Jazz
4. Stay Woke (feat. Miguel) (Meek Mill) --> 1: Pop, 2: Rap, 3: EDM, Aux: Bluegrass
5. Plain Jane (A$AP Ferg) --> 1: Hip Hop, 2: Rap, 3: Pop, Aux: Metal
6. X (with 2 Chainz & Saudi) (ScHoolboy Q) --> 1: Hip Hop, 2: Rap, 3: Pop, Aux: Metal
7. See You Again (feat. Charlie Puth) (Wiz Khalifa) --> 1: Pop, 2: EDM, 3: Rap, Aux

give me a ska version of 'turning japanese'
give ma classical version of 'x'
search song by title and then run through genre prediction